# MIS-CPIMS Data Import Scripts

Import pandas library and read exported CPARA data from MIS (XLSX). Display all columns for exploration.

In [3]:
import pandas as pd
# Reads USAID 4The Child cpara data
cpara=pd.read_excel("Data/CPARA.xlsx", sheet_name="Sheet1")
# cpara.columns

Index(['UnIque ID', 'record_id', 'assessment_date', 'cpims_id', 'case_manager',
       'child_headed', 'has_hei', 'has_pbf', 'has_svac', 'q1_1', 'q1_2',
       'q1_3', 'q1_4', 'benchmark1', 'q2_1', 'q2_2', 'q2_3', 'q2_4', 'q2_5',
       'q2_6', 'q2_8', 'q2_7', 'q2_9', 'benchmark2', 'q3_1', 'q3_2', 'q3_3',
       'benchmark3', 'q4_1', 'q4_2', 'q4_3', 'q4_4', 'benchmark4', 'q5_1',
       'q5_2', 'q5_3', 'benchmark5', 'q6_1', 'q6_2', 'q6_3', 'q6_4', 'q6_5',
       'benchmark6', 'q7_1', 'q7_2', 'benchmark7', 'q8_1', 'benchmark8',
       'q9_1', 'q9_2', 'q9_3', 'q9_4', 'benchmark9', 'healthy', 'stable',
       'safe', 'schooled', 'entry_time', 'action_by', 'designation', 'lip',
       'entry_by_name', 'client_time', 'deleted', 'modified', 'gps', 'device',
       'cpimsId', 'names', 'caregiverhivstatus', 'cbo_id', 'cbo', 'county',
       'subcounty', 'ward', 'chv_id', 'chv'],
      dtype='object')

In [30]:
# reads the mapping for U4TC mapping for HH_id, cpims_ovc_id, caregiver id
cpims_hhs=pd.read_csv("Data/Household ID OVC Mapping.csv")

In [ ]:
# Reads individual cpara data for U4TC
cpara_ovc_questions = pd.read_excel("Data/OVC_Questions.xlsx", sheet_name="Sheet1")

In [6]:
# Generate UID whose length is equivalent to the U4TC cpara data
import uuid

record_counts = cpara['record_id'].count()

def generate_uid(count):
    uid_list = []
    for i in range (0, count):
        uuid_gen = uuid.uuid4()
        # print(f'generated uid: {uuid_gen} inte: {i}')
        uid_list.append(uuid_gen)
    return uid_list    

df_uid = pd.DataFrame(generate_uid(record_counts))

In [58]:
# Reads the cpims questions mapping
cpims_questions=pd.read_excel("Data/ovc_care_questions.xlsx")
cpims_questions.columns

Index(['question_id', 'code', 'question', 'domain', 'question_text',
       'question_type', 'is_void', 'timestamp_created', 'timestamp_updated',
       'form_id', 'CURRENT cPARA', 'ind_cpara', 'cpara'],
      dtype='object')

In [31]:
# Mergers cpara data with household id, cpims id on caregiver_id
cpara_data1=pd.merge(cpara, cpims_hhs, how='left', left_on='cpims_id', right_on='caregiver_id')
# Merges cpara data with the uids
cpara_data = pd.merge(cpara_data1, df_uid, left_index=True, right_index=True)
cpara_data.head(2)

,UnIque ID,record_id,assessment_date,cpims_id,case_manager,child_headed,has_hei,has_pbf,has_svac,q1_1,...,county,subcounty,ward,chv_id,chv,Unnamed: 0,cpims_ovc_id,caregiver_id,household,0
0,2022-09-12-105151,9325,2022-09-12,105151,NaN,No,No,No,No,Yes,...,Kisumu,Kisumu East,Manyatta 'B',3173240.0,GILLIANN GILLIAN ACHIENGGG,99003.0,4282788.0,105151.0,29d199db-2f32-4025-bb9b-94357558550b,16142843-84ab-4cbe-9edd-de02ac81eef9
1,2022-09-12-105151,9325,2022-09-12,105151,NaN,No,No,No,No,Yes,...,Kisumu,Kisumu East,Manyatta 'B',3173240.0,GILLIANN GILLIAN ACHIENGGG,99026.0,4282807.0,105151.0,29d199db-2f32-4025-bb9b-94357558550b,785a1f6b-6301-40dd-ac67-72b5f059d121


In [32]:
# cpara individual table merges with with household id, cpims id on caregiver_id
cpara_individual=pd.merge(cpara_ovc_questions, cpims_hhs, how='left', left_on='caregiver_id', right_on='caregiver_id')
cpara_individual.head(2)

,record_id,cpara_record_id,cpara_date,question,response,cpims_id,caregiver_id,chv_id,Unnamed: 0,cpims_ovc_id,household
0,5,8859,2022-08-02,q3.1,Yes,1705501,1676623,2042693,43178.0,1687650.0,b540b485-7399-4407-bd14-066f5b79c133
1,5,8859,2022-08-02,q3.1,Yes,1705501,1676623,2042693,43184.0,1705501.0,b540b485-7399-4407-bd14-066f5b79c133


In [5]:
cpara_ovc_questions.dtypes #cpims_hhs.dtypes

record_id           int64
cpara_record_id     int64
cpara_date         object
question           object
response           object
cpims_id            int64
caregiver_id        int64
chv_id              int64
dtype: object

Align the MIS data from rows to columns => vertical and only display the first two records. Move the columns that should not be considered as var.

In [33]:
# converts the horizontal table of cpara data to vertical table for manipulation
cpara_unpivot=pd.melt(cpara_data, id_vars=['record_id', 'assessment_date', 'cpims_id', 'entry_time', 'action_by',
       'designation', 'lip', 'entry_by_name','client_time', 'modified', 'deleted', 'gps', 'device', 'case_manager', 'cpims_ovc_id', 'household', 0])

cpara_unpivot.head(2)

,record_id,assessment_date,cpims_id,entry_time,action_by,designation,lip,entry_by_name,client_time,modified,deleted,gps,device,case_manager,cpims_ovc_id,household,0,variable,value
0,9325,2022-09-12,105151,2022-09-27 12:49:56,3180741,Data Assistant,MAKE ME SMILE,ZILPA AKINYI,2022-09-27 13:49:49,2022-09-27 12:49:56,NaN,NaN,screen=1440 x 900;browser=Chrome;browserVersio...,NaN,4282788.0,29d199db-2f32-4025-bb9b-94357558550b,16142843-84ab-4cbe-9edd-de02ac81eef9,UnIque ID,2022-09-12-105151
1,9325,2022-09-12,105151,2022-09-27 12:49:56,3180741,Data Assistant,MAKE ME SMILE,ZILPA AKINYI,2022-09-27 13:49:49,2022-09-27 12:49:56,NaN,NaN,screen=1440 x 900;browser=Chrome;browserVersio...,NaN,4282807.0,29d199db-2f32-4025-bb9b-94357558550b,785a1f6b-6301-40dd-ac67-72b5f059d121,UnIque ID,2022-09-12-105151


In [12]:
cpara_unpivot.columns

Index(['record_id', 'assessment_date', 'cpims_id', 'entry_time', 'action_by',
       'designation', 'lip', 'entry_by_name', 'client_time', 'modified',
       'deleted', 'gps', 'device', 'case_manager', 'cpims_ovc_id', 'household',
       'variable', 'value'],
      dtype='object')

In [43]:
# filters bencmark questions and pivots them.
ovc_care_benchmark = cpara_unpivot[cpara_unpivot["variable"].str.contains('benchmark')]#.pivot(index=['record_id', 'cpims_id','assessment_date','entry_time', 'modified', 'cpims_ovc_id', 'household', 0], columns='variable', values=['value'])
# ovc_care_benchmark.to_excel("ovc_care_benchmark.xlsx")
unique_hh = ovc_care_benchmark['household'].str.contains('NaN')

# print(unique_hh.str.contains('NaN'))

In [45]:
unique_hh.describe()

count     293004
unique         1
top        False
freq      293004
Name: household, dtype: object

In [50]:
# Filter cpara question and pivots them or not.  // send to excel
#ovc_care_cpara = cpara_unpivot[cpara_unpivot["variable"].str.startswith('q')]
ovc_care_cpara = cpara_unpivot[cpara_unpivot["variable"].str.startswith('q')]#.pivot(index=['record_id', 'cpims_id','assessment_date','cpims_ovc_id', 'household'], columns='variable', values=['value'])
# ovc_care_cpara.to_csv("ovc_care_cpara_pivot.csv")

record_id
assessment_date
cpims_id
entry_time
action_by
designation
lip
entry_by_name
client_time
modified
deleted
gps
device
case_manager
cpims_ovc_id
household
0
variable
value


In [52]:
cpims_questions.columns

Index(['question_id', 'code', 'question', 'domain', 'question_text',
       'question_type', 'is_void', 'timestamp_created', 'timestamp_updated',
       'form_id', 'CURRENT cPARA', 'USAID 4THECHILD QID'],
      dtype='object')

In [61]:
ovc_cpara_with_q = pd.merge(ovc_care_cpara, cpims_questions, how='left', left_on='variable', right_on='cpara')
for i in ovc_cpara_with_q.columns:
    print(i)
ovc_cpara_with_q.columns

record_id
assessment_date
cpims_id
entry_time
action_by
designation
lip
entry_by_name
client_time
modified
deleted
gps
device
case_manager
cpims_ovc_id
household
0
variable
value
question_id
code
question
domain
question_text
question_type
is_void
timestamp_created
timestamp_updated
form_id
CURRENT cPARA
ind_cpara
cpara


Index([        'record_id',   'assessment_date',          'cpims_id',
              'entry_time',         'action_by',       'designation',
                     'lip',     'entry_by_name',       'client_time',
                'modified',           'deleted',               'gps',
                  'device',      'case_manager',      'cpims_ovc_id',
               'household',                   0,          'variable',
                   'value',       'question_id',              'code',
                'question',            'domain',     'question_text',
           'question_type',           'is_void', 'timestamp_created',
       'timestamp_updated',           'form_id',     'CURRENT cPARA',
               'ind_cpara',             'cpara'],
      dtype='object')

In [10]:
cpara_individual.to_excel("cpara_ovc_questions.xlsx")
cpara_individual.head(2)

,record_id,cpara_record_id,cpara_date,question,response,cpims_id,caregiver_id,chv_id,cpims_ovc_id,household
0,5,8859,2022-08-02,q3.1,Yes,1705501,1676623,2042693,1687650.0,b540b485-7399-4407-bd14-066f5b79c133
1,5,8859,2022-08-02,q3.1,Yes,1705501,1676623,2042693,1705501.0,b540b485-7399-4407-bd14-066f5b79c133


In [11]:
cpims_hhs.columns

Index(['cpims_ovc_id', 'caregiver_id', 'household'], dtype='object')

In [108]:
cpara_data.to_excel("CPARA_MAPPING.xlsx")

In [73]:
cpara_data_output = ovc_cpara_with_q.rename(columns={"code": "question_code","value": "answer","question_type": "question_type","domain": "domain","assessment_date": "date_of_event","client_time": "timestamp_created","cpims_id": "caregiver_id",0:  "event_id","household": "household_id","cpims_ovc_id": " person_id","question_id": "question_id"})

cpara_data_output['cpara_id'] = ''
cpara_data_output['date_of_previous_event'] = ''
cpara_data_output['is_void'] = ''
# cpara_data_output['timestamp_updated'] = ''


cpara_data_output.columns
cpara_output_sql = cpara_data_output["cpara_id","question_code","answer","question_type","domain","date_of_event","date_of_previous_event","timestamp_created","is_void","timestamp_updated","caregiver_id","event_id","household_id","person_id","question_id"]

cpara_output_sql


KeyError: ('cpara_id', 'question_code', 'answer', 'question_type', 'domain', 'date_of_event', 'date_of_previous_event', 'timestamp_created', 'is_void', 'timestamp_updated', 'caregiver_id', 'event_id', 'household_id', 'person_id', 'question_id')